In [ ]:
# Poisson problem on an L-shape domain with Dirichlet boundary conditions
# This problem solves Example 1.1.2 of the book of Elman et al.

import numpy as np
import eztfem as ezt
from func import func
from scipy.sparse.linalg import spsolve


In [ ]:
# create mesh
mesh = ezt.l_shape2d([20, 20, 20, 20], 'quad4',
                     factor=np.array([10, 10, 10, 10]))


In [ ]:
# define the problem
elementdof = np.array([[1, 1, 1, 1],
                       [2, 2, 2, 2]], dtype=int).transpose()
problem = ezt.Problem(mesh, elementdof, nphysq=1)


In [ ]:
# define Gauss integration and basis functions
user = ezt.User()
shape = 'quad'
user.xr, user.wg = ezt.gauss_legendre(shape, n=2)
user.phi, user.dphi = ezt.basis_function(shape, 'Q1', user.xr)


In [ ]:
# user struct for setting problem coefficients, ...
user.coorsys = 0
user.alpha = 1
user.funcnr = 5
user.func = func


In [ ]:
# assemble the system matrix and vector
A, f = ezt.build_system(mesh, problem, ezt.poisson_elem, user)


In [ ]:
# define essential boundary conditions (Dirichlet)
iess = ezt.define_essential(mesh, problem, 'curves',
                            [0, 1, 2, 3, 4, 5, 6, 7])


In [ ]:
# fill values for the essential boundary conditions
uess = np.zeros([problem.numdegfd])


In [ ]:
# apply essential boundary conditions to the system
ezt.apply_essential(A, f, uess, iess)


In [ ]:
# solve the system
u = spsolve(A.tocsr(), f)


In [ ]:
# maximum value
print('Maximum value ', np.max(u))


In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)


In [ ]:
# plot the solution field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400))
